In [1]:
from keras.models import load_model
from SiameseModel import SiameseModel
from SiameseLoader import SiameseLoader
import glob
import os
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)

Using TensorFlow backend.


# Load data set

In [2]:
loader = SiameseLoader('./data')

loading snake breed: 152
loading snake breed: 159
loading snake breed: 80
loading snake breed: 187
loading snake breed: 3


/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


loading snake breed: 46
loading snake breed: 158
loading snake breed: 87
loading snake breed: 123
loading snake breed: 100
loading snake breed: 205
loading snake breed: 197
loading snake breed: 120
loading snake breed: 61
loading snake breed: 124
loading snake breed: 67
loading snake breed: 110
loading snake breed: 44
loading snake breed: 203
loading snake breed: 32
loading snake breed: 43
loading snake breed: 111
loading snake breed: 219
loading snake breed: 156
loading snake breed: 175
loading snake breed: 193
loading snake breed: 201
loading snake breed: 88
loading snake breed: 83
loading snake breed: 109
loading snake breed: 107
loading snake breed: 185
loading snake breed: 89
loading snake breed: 206
loading snake breed: 165
loading snake breed: 127
loading snake breed: 15
loading snake breed: 230
loading snake breed: 204
loading snake breed: 21
loading snake breed: 66
loading snake breed: 167
loading snake breed: 63
loading snake breed: 128
loading snake breed: 45
loading snake b

# Load siamese network

In [3]:
ckpts = glob.glob("checkpoints/*.hdf5")
latest_ckpt = max(ckpts, key=os.path.getctime)
print("loading from checkpoint: ", latest_ckpt)
initial_epoch = int(latest_ckpt[latest_ckpt.find("-epoch-") + len("-epoch-"):latest_ckpt.rfind("-lr-")])
model = load_model(latest_ckpt)

loading from checkpoint:  checkpoints/flowchroma-epoch-00270-lr--train_loss-0.6375-val_loss-0.6800.hdf5


/usr/local/lib/python3.5/dist-packages/keras/engine/topology.py:1271: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


# Accuracy function 

In [5]:
def find_accuracy(df):
    correct =  df.loc[df['predictions'] == df['targets']].shape[0]
    incorrect = df.loc[df['predictions'] != df['targets']].shape[0]
    total = df.shape[0]
    print('Accuracy : {}%'.format(100*correct/total))
    return correct, incorrect, total
    

# Test on validation set (Test 1)

In [53]:
pairs, targets = loader.get_validation_batch(50)
prediction = model.predict(pairs)

#data post processing
df = pd.DataFrame(data={'predictions':prediction.reshape(prediction.shape[0]), 'targets':targets.reshape(targets.shape[0])})
df.loc[df['predictions'] >= 0.5, 'predictions'] = 1
df.loc[df['predictions'] < 0.5, 'predictions'] = 0
find_accuracy(df)


Accuracy : 52.0%


(26, 24, 50)

# Test for real classification of image (Test 2)

In [81]:
pairs, targets = loader.make_oneshot_task(2)
prediction = model.predict(pairs)
df = pd.DataFrame(data={'prediction':prediction.reshape(prediction.shape[0]), 'targets':targets.reshape(targets.shape[0])})


84 2


In [ ]:
loader.

In [16]:
prediction = model.predict(pairs)

In [17]:
df = pd.DataFrame(data={'prediction':prediction.reshape(prediction.shape[0]), 'targets':targets.reshape(targets.shape[0])})

In [18]:
df.loc[df['prediction'] > 0.5, 'prediction'] = 1
df.loc[df['prediction'] < 0.5, 'prediction'] = 0
df.head(800)

,prediction,targets
0,1.0,0.0
1,0.0,0.0
2,0.0,1.0
3,0.0,0.0
4,0.0,0.0
5,0.0,0.0
6,0.0,0.0
7,0.0,0.0
8,0.0,0.0
9,0.0,0.0


In [20]:
df.loc[df['prediction']!=df['targets']].head(400)

,prediction,targets
0,1.0,0.0
2,0.0,1.0
30,1.0,0.0
33,1.0,0.0
35,1.0,0.0
36,1.0,0.0
42,1.0,0.0
47,1.0,0.0
53,1.0,0.0
60,1.0,0.0
